### Run pipeline to remove duplicates from table locations and standardize US state names

In [1]:
%run locations_dataflow.py

/home/jupyter/venv/lib/python3.7/site-packages/apache_beam/io/gcp/bigquery.py:1936: BeamDeprecationWarning: options is deprecated since First stable release. References to <pipeline>.options will not be supported
  temp_location = pcoll.pipeline.options.view_as(
/home/jupyter/venv/lib/python3.7/site-packages/apache_beam/io/gcp/bigquery_file_loads.py:956: BeamDeprecationWarning: options is deprecated since First stable release. References to <pipeline>.options will not be supported
  temp_location = p.options.view_as(GoogleCloudOptions).temp_location


#### Check that only duplicates were removed

In [2]:
%%bigquery
SELECT count(*)+(SELECT count(*)
                FROM datamart.locations
                WHERE latitude = 0 or longitude =0)-1+(SELECT count(*)
                                                        FROM datamart.locations
                                                        WHERE latitude IS null or longitude IS null)-1 AS distinct_locations,
                (SELECT count(*)
                FROM datamart.locations_Dataflow) AS locations_Dataflow_count
FROM (SELECT DISTINCT  latitude, longitude FROM datamart.locations)

,distinct_locations,locations_Dataflow_count
0,4305,4305


#### PK check: no duplicates if same number

In [3]:
%%bigquery
SELECT count(*) AS distinct_PK, (SELECT count(*) FROM datamart.locations_Dataflow) AS locations_count
FROM (
    SELECT DISTINCT id
    FROM datamart.locations_Dataflow
)

,distinct_PK,locations_count
0,4305,4305


### SQL Transforms on locations_Dataflow

#### Ensure that abbreviated US state names have all been standardized to the full name

In [4]:
%%bigquery
SELECT distinct state, count(*) AS count
FROM datamart.locations_Dataflow
WHERE country='US'
GROUP BY state
ORDER BY state

,state,count
0,None,1
1,Alabama,71
2,Alaska,37
3,American Samoa,1
4,Arizona,22
...,...,...
64,Washington,53
65,West Virginia,57
66,Wisconsin,76
67,Wuhan Evacuee,1


In [5]:
%%bigquery
UPDATE datamart.locations_Dataflow
SET state='District of Columbia'
WHERE state='D.C.'

""


In [6]:
%%bigquery
UPDATE datamart.locations_Dataflow
SET state='Texas', combined_key='Diamond Princess, Lackland, Texas, US'
WHERE state='TX (From Diamond Princess)'

""


In [7]:
%%bigquery
UPDATE datamart.locations_Dataflow
SET state='Nebraska', combined_key='Diamond Princess, Omaha, Nebraska, US'
WHERE state='NE (From Diamond Princess)'

""


In [8]:
%%bigquery
UPDATE datamart.locations_Dataflow
SET state='California', combined_key='Diamond Princess, Travis, California, US'
WHERE state='CA (From Diamond Princess)'

""


In [9]:
%%bigquery
SELECT distinct state, count(*) AS count
FROM datamart.locations_Dataflow
WHERE country='US'
GROUP BY state
ORDER BY state

,state,count
0,None,1
1,Alabama,71
2,Alaska,37
3,American Samoa,1
4,Arizona,22
...,...,...
60,Washington,53
61,West Virginia,57
62,Wisconsin,76
63,Wuhan Evacuee,1


#### PK check: no duplicates if same number

In [10]:
%%bigquery
SELECT count(*) AS distinct_PK, (SELECT count(*) FROM datamart.locations_Dataflow) AS locations_count
FROM (
    SELECT DISTINCT id
    FROM datamart.locations_Dataflow
)

,distinct_PK,locations_count
0,4305,4305


### SQL Transforms on cases (milestone 2's version)

### Remake milestone 2's cases table since it was modified in milestone 3

#### create preliminary covid table

In [11]:
%%bigquery
CREATE OR REPLACE TABLE datamart.covid AS
SELECT null AS id, fips, admin2, province_state, country_region AS Country, last_update, latitude, longitude, confirmed, deaths, recovered, active, combined_key, incident_rate, case_fatality_ratio
FROM jhu_daily_reports_staging.covid_winter2020
UNION ALL
SELECT null AS id, fips, admin2, province_state, country_region AS Country, last_update, latitude, longitude, confirmed, deaths, recovered, active, combined_key, incident_rate, case_fatality_ratio
FROM jhu_daily_reports_staging.covid_spring2020
UNION ALL
SELECT null AS id, fips, admin2, province_state, country_region AS Country, CAST(last_update AS STRING) AS last_update, lat AS latitude, long_ AS longitude, confirmed, deaths, recovered, active, combined_key, incidence_rate AS incident_rate, case_fatality_ratio
FROM jhu_daily_reports_staging.covid_summer2020
UNION ALL
SELECT null AS id, fips, admin2, province_state, country_region AS Country, CAST(last_update AS STRING) AS last_update, lat AS latitude, long_ AS longitude, confirmed, deaths, recovered, active, combined_key, incident_rate, case_fatality_ratio
FROM jhu_daily_reports_staging.covid_fall2020
UNION ALL
SELECT null AS id, fips, admin2, province_state, country_region AS Country, CAST(last_update AS STRING) AS last_update, lat AS latitude, long_ AS longitude, confirmed, deaths, recovered, active, combined_key, incident_rate, case_fatality_ratio
FROM jhu_daily_reports_staging.covid_winter2021
UNION ALL
SELECT null AS id, fips, admin2, province_state, country_region AS Country, CAST(last_update AS STRING) AS last_update, lat AS latitude, long_ AS longitude, confirmed, deaths, recovered, active, combined_key, incident_rate, case_fatality_ratio
FROM jhu_daily_reports_staging.covid_spring2021

""


#### compute fingerprint ids for preliminary covid table

In [12]:
%%bigquery
UPDATE datamart.covid
SET id=FARM_FINGERPRINT(country)
WHERE province_state IS null

""


In [13]:
%%bigquery
UPDATE datamart.covid
SET id=FARM_FINGERPRINT(concat(province_state, country))
WHERE (province_state IS NOT null AND country!='US')

""


In [14]:
%%bigquery
UPDATE datamart.covid
SET id=FARM_FINGERPRINT(concat(admin2, province_state, country))
WHERE (province_state IS NOT null AND country='US' AND admin2 IS NOT null AND fips IS NULL)

""


In [15]:
%%bigquery
UPDATE datamart.covid
SET id=FARM_FINGERPRINT(concat(CAST(fips AS STRING), admin2, province_state, country))
WHERE (province_state IS NOT null AND country='US' AND admin2 IS NOT null AND fips IS NOT null)

""


In [16]:
%%bigquery
UPDATE datamart.covid
SET id=FARM_FINGERPRINT(concat(CAST(fips AS STRING), province_state, country))
WHERE (province_state IS NOT null AND country='US' AND admin2 IS null AND fips IS NOT null)

""


In [17]:
%%bigquery
UPDATE datamart.covid
SET id=FARM_FINGERPRINT(concat(province_state, country))
WHERE (province_state IS NOT null AND country='US' AND admin2 IS null AND fips IS null)

""


#### check that there is no null id field in the preliminary covid table

In [18]:
%%bigquery
SELECT count(*) AS null_id_count
FROM datamart.covid
WHERE id IS null

,null_id_count
0,0


#### create Cases_raw table from preliminary covid table

In [20]:
%%bigquery
CREATE OR REPLACE TABLE datamart.Cases_raw AS
SELECT id AS location_id, last_update, confirmed, deaths, recovered, active, incident_rate, case_fatality_ratio
FROM datamart.covid

""


#### create Cases_hypen_date_fixed

In [21]:
%%bigquery
CREATE OR REPLACE TABLE datamart.Cases_hyphen_date_fixed AS
SELECT location_id, CAST(last_update AS DATETIME) AS last_update, confirmed, deaths, recovered, active, incident_rate, case_fatality_ratio
FROM datamart.Cases_raw
WHERE strpos(last_update, '-') > 0 AND strpos(last_update, ' ') > 0 AND strpos(last_update, '+') = 0

""


#### create Cases_plus_date_fixed

In [22]:
%%bigquery
CREATE OR REPLACE TABLE datamart.Cases_plus_date_fixed AS
SELECT location_id,CAST(CAST(last_update AS TIMESTAMP) AS DATETIME) AS last_update, confirmed, deaths, recovered, active, incident_rate, case_fatality_ratio
FROM datamart.Cases_raw
WHERE strpos(last_update, '-') > 0 AND strpos(last_update, ' ') > 0 AND strpos(last_update, '+') > 0

""


#### create Cases_slash_date

In [23]:
%%bigquery
CREATE OR REPLACE TABLE datamart.Cases_slash_date AS
SELECT * 
FROM datamart.Cases_raw
WHERE strpos(last_update, '/') > 0

""


#### create Cases_slash_date_fixed

In [24]:
%%bigquery
CREATE OR REPLACE TABLE datamart.Cases_slash_date_fixed AS
(SELECT location_id, PARSE_DATETIME('%m/%d/%y %H:%M', last_update) AS Last_Update, confirmed, deaths, recovered, active, incident_rate, case_fatality_ratio
FROM datamart.Cases_slash_date 
WHERE length(split(split(last_update, ' ')[offset(0)], '/')[offset(2)]) = 2
UNION ALL
SELECT location_id, PARSE_DATETIME('%m/%d/%Y %H:%M', last_update) AS Last_Update, confirmed, deaths, recovered, active, incident_rate, case_fatality_ratio
FROM datamart.Cases_slash_date 
WHERE length(split(split(last_update, ' ')[offset(0)], '/')[offset(2)]) = 4
)

""


#### create cases table

In [25]:
%%bigquery
CREATE OR REPLACE TABLE datamart.cases AS
(SELECT * 
FROM datamart.Cases_hyphen_date_fixed
UNION ALL
SELECT *
FROM datamart.Cases_plus_date_fixed
UNION ALL
SELECT *
FROM datamart.Cases_slash_date_fixed
)

""


#### remove duplicate cases

In [26]:
%%bigquery
CREATE OR REPLACE TABLE datamart.cases AS
SELECT DISTINCT *
FROM datamart.cases

""


#### check if there a still duplicate cases

In [27]:
%%bigquery
SELECT location_id, last_update, count(*) AS duplicate_cases
FROM datamart.cases
GROUP BY location_id, last_update
HAVING count(*) > 1
ORDER BY count(*) DESC
LIMIT 10

,location_id,last_update,duplicate_cases
0,3183797238637395852,2021-04-02 15:13:53,146
1,-1181716864917925688,2021-04-02 15:13:53,111
2,2231787937947570706,2020-12-21 13:27:30,92
3,8778414404485170876,2020-03-17 11:53:10,3
4,8778414404485170876,2020-02-24 23:33:02,3
5,5908437069604758697,2020-03-14 11:53:29,2
6,8009782315755301025,2020-08-04 02:27:56,2
7,-7786566325262141720,2020-12-21 13:27:30,2
8,7430223413705102985,2020-03-14 20:13:16,2
9,7612477503619687113,2020-01-31 10:37:00,2


#### remove duplicates based on confirmed, deaths, and recovered cases

In [28]:
%%bigquery
CREATE OR REPLACE TABLE datamart.cases AS
SELECT location_id, last_update, confirmed, deaths, recovered, active, incident_rate, case_fatality_ratio
FROM(SELECT *, rank() over (partition by location_id, last_update order by confirmed DESC, deaths DESC, recovered DESC ) as rank
FROM datamart.cases)
WHERE rank = 1

""


#### PK check: if same number then no duplicates

In [29]:
%%bigquery
SELECT count(*) AS distinct_PK, (SELECT count(*) FROM datamart.cases) AS cases_count
FROM (
    SELECT DISTINCT location_id, last_update
    FROM datamart.cases
)

,distinct_PK,cases_count
0,1475767,1475767


#### FK check

In [30]:
%%bigquery
SELECT count(*) AS foreign_key_violations
FROM datamart.cases AS c
LEFT JOIN datamart.locations AS l ON c.location_id=l.id
WHERE l.id IS null

,foreign_key_violations
0,0


#### make a copy of the milestone 2 cases table to modify: new_cases

In [11]:
%%bigquery
CREATE OR REPLACE TABLE datamart.new_cases AS
SELECT * 
FROM datamart.cases

""


### SQL Transforms: Update location_id of the orphaned records in table new_cases

#### get the abandoned location records {review!

In [12]:
%%bigquery
CREATE OR REPLACE TABLE datamart.abandoned_locations AS
SELECT *
FROM datamart.locations AS l
WHERE l.id NOT IN (SELECT d.id FROM datamart.locations_Dataflow AS d)

""


In [53]:
## don't run, may not need
%%bigquery
CREATE OR REPLACE TABLE datamart.abandoned_locations AS
SELECT a.location_id AS abandoned_id, l.latitude, l.longitude
FROM datamart.abandoned_locations AS a
JOIN datamart.locations AS l ON l.id=a.location_id

""


#### Create a junction table to match abandoned ids to kept ids

In [13]:
%%bigquery
SELECT l.id AS kept_id, a.id AS rid_id, l.latitude, l.longitude
FROM datamart.locations_Dataflow AS l
JOIN datamart.abandoned_locations AS a 
ON l.latitude=a.latitude AND l.longitude=a.longitude
ORDER BY kept_id, rid_id
LIMIT 10

,kept_id,rid_id,latitude,longitude
0,-8869782645955986065,-3845568702501865415,46.230401,-118.477554
1,-8673092529043857217,6005352482531411353,-51.796300,-59.523600
2,-8054612975524920966,8624219087402172771,31.825700,117.226400
3,-6976893411093504258,-4147227804562828434,13.443200,-15.310100
4,-6843305877291741994,-1909288589590523291,36.140800,-5.353600
5,-6369225154884557074,24401797468677984,27.614000,115.722100
6,-6190303266847582575,-2262583433077819139,40.767273,-73.971526
7,-6125909817942919488,-8744039356309243705,15.097900,145.673900
8,-6125909817942919488,3620381022065474036,15.097900,145.673900
9,-5555958697111529338,5491571796867280413,40.712800,-74.006000


In [14]:
%%bigquery
CREATE OR REPLACE TABLE datamart.junction AS
SELECT l.id AS kept_id, a.id AS rid_id, l.latitude, l.longitude
FROM datamart.locations_Dataflow AS l
JOIN datamart.abandoned_locations AS a 
ON l.latitude=a.latitude AND l.longitude=a.longitude

""


In [15]:
%%bigquery
UPDATE datamart.new_cases AS c
SET c.location_id=(SELECT j.kept_id FROM datamart.junction AS j WHERE c.location_id=j.rid_id)
WHERE c.location_id IN (SELECT ju.rid_id FROM datamart.junction AS ju)

""


#### PK check: no duplicates if same number

In [16]:
%%bigquery
SELECT count(*) AS distinct_PK, (SELECT count(*) FROM datamart.new_cases) AS cases_count
FROM (
    SELECT DISTINCT location_id, last_update
    FROM datamart.new_cases
)

,distinct_PK,cases_count
0,1475736,1475767


#### remove duplicates based on confirmed, deaths, and recovered cases

In [17]:
%%bigquery
CREATE OR REPLACE TABLE datamart.new_cases AS
SELECT location_id, last_update, confirmed, deaths, recovered, active, incident_rate, case_fatality_ratio
FROM(SELECT *, rank() over (partition by location_id, last_update order by confirmed DESC, deaths DESC, recovered DESC ) as rank
FROM datamart.new_cases)
WHERE rank = 1

""


#### PK re-check: no duplicates if same number

In [18]:
%%bigquery
SELECT count(*) AS distinct_PK, (SELECT count(*) FROM datamart.new_cases) AS cases_count
FROM (
    SELECT DISTINCT location_id, last_update
    FROM datamart.new_cases
)

,distinct_PK,cases_count
0,1475736,1475737


##### investigate duplicate case

In [19]:
%%bigquery
SELECT location_id, last_update, count(*) AS duplicate_cases
FROM datamart.new_cases
GROUP BY location_id, last_update
HAVING count(*) > 1

,location_id,last_update,duplicate_cases
0,-278257842025680577,2020-03-08 17:33:03,2


In [20]:
%%bigquery
SELECT *
FROM datamart.new_cases
WHERE location_id=-278257842025680577 AND last_update='2020-03-08 17:33:03'

,location_id,last_update,confirmed,deaths,recovered,active,incident_rate,case_fatality_ratio
0,-278257842025680577,2020-03-08 17:33:03,6,0,0,NaN,NaN,NaN
1,-278257842025680577,2020-03-08 17:33:03,6,0,0,NaN,NaN,NaN


##### remove duplicate by SELECT DISTINCT: the single duplicate record is an exact copy that cannot be ranked

In [21]:
%%bigquery
CREATE OR REPLACE TABLE datamart.new_cases AS
SELECT DISTINCT *
FROM datamart.new_cases

""


#### PK re-re-check: no duplicates if same number

In [22]:
%%bigquery
SELECT count(*) AS distinct_PK, (SELECT count(*) FROM datamart.new_cases) AS cases_count
FROM (
    SELECT DISTINCT location_id, last_update
    FROM datamart.new_cases
)

,distinct_PK,cases_count
0,1475736,1475736


#### FK check

In [23]:
%%bigquery
SELECT count(*) AS foreign_key_violations
FROM datamart.new_cases AS c
LEFT JOIN datamart.locations_Dataflow AS l ON c.location_id=l.id
WHERE l.id IS null

,foreign_key_violations
0,0


#### Replace cases with new_cases' data

In [1]:
%%bigquery
CREATE OR REPLACE TABLE datamart.cases AS
SELECT *
FROM datamart.new_cases

""


### 3 SQL Queries

#### Display the population of Texans affected by COVID-19 based on the average incident rate of the most recent updated Texan cases, and distribute the population by the gross rent they pay. (Assuming that all Texans occupy housing units that require rent, that the percentages in the housing table is applicable to Texans as is, and that the total Texas population is 8.74% of the US's total population)

In [3]:
%%bigquery
SELECT label AS Gross_Rent, (h.percent/100)*
                                (((SELECT AVG(incident_rate)
                                    FROM datamart.cases 
                                    WHERE last_update=(SELECT MAX(last_update) 
                                                        FROM datamart.cases AS c 
                                                        JOIN datamart.locations_Beam AS l ON c.location_id=l.id 
                                                        WHERE l.state='Texas' AND l.country='US'))/100000)
                                *
                                ((SELECT estimate FROM datamart.demographics WHERE label='Total population')*8.74/100)) AS Estimated_COVID_Affected_Population 
FROM datamart.housing AS h
WHERE subcategory='gross rent' AND percent IS NOT null
ORDER BY Gross_Rent

,Gross_Rent,Estimated_COVID_Affected_Population
0,"$1,000 to $1,499",715547.538817
1,"$1,500 to $1,999",333922.184781
2,"$2,000 to $2,499",133568.873912
3,"$2,500 to $2,999",57243.803105
4,"$3,000 or more",57243.803105
5,$500 to $999,863427.363506
6,Less than $500,224204.895496


#### Display the estimated number of confirmed COVID-19 cases during January of 2021 per current educational status (those in school) as well as highest educational attainment (those out of school), ordered by ascending number of confimed cases

In [20]:
%%bigquery
SELECT label AS Enrollment, (s.percent/100)*
                                (SELECT SUM(Confirmed)
                                    FROM datamart.cases
                                     WHERE last_update between '2021-01-01' AND '2021-01-31') AS Estimated_COVID_Confirmed_Cases
FROM datamart.social AS s
WHERE (subcategory='School Enrollment' OR subcategory = 'Educational Attainment') AND percent IS NOT null
ORDER BY Estimated_COVID_Confirmed_Cases

,Enrollment,Estimated_COVID_Confirmed_Cases
0,Kindergarten,139704881.200000000
1,Less than 9th grade,142498978.824000000
2,"9th to 12th grade, no diploma",192792736.056000000
3,Associate's degree,237498298.040000000
4,Graduate or professional degree,346468105.376000000
5,Bachelor's degree,553231329.552000000
6,"Some college, no degree",569995915.296000000
7,High school (grades 9-12),586760501.040000000
8,High school graduate (includes equivalency),754406358.480000000
9,College or graduate school,768376846.600000000


#### Display the estimated number of deaths per age group in the US during 2021 (up until April 14, 2021), ordered by estimated deaths in descending order and limited by 10

In [26]:
%%bigquery
SELECT d.label AS Age_Group, (d.percent/100)*
                                        (SELECT SUM(deaths)
                                        FROM datamart.cases AS c
                                        JOIN datamart.locations AS l ON c.location_id=l.id
                                        WHERE l.country='US' AND last_update between '2021-01-01' and '2021-04-14 23:59:59') AS Estimated_Deaths
                                    
FROM datamart.demographics as d
WHERE (subcategory1 = 'age' AND subcategory2 is null AND d.percent is not null)
ORDER BY Estimated_Deaths DESC LIMIT 10;

,Age_Group,Estimated_Deaths
0,16 years and over,40238436.800000000
1,21 years and over,36868467.718000000
2,Under 18 years,11367358.396000000
3,62 years and over,9657224.832000000
4,25 to 34 years,6991428.394000000
5,45 to 54 years,6538745.980000000
6,35 to 44 years,6337553.796000000
7,65 to 74 years,4577122.186000000
8,20 to 24 years,3420267.128000000
9,55 to 59 years,3369969.082000000


### Create Views from Above SQL Queries

In [1]:
%%bigquery
CREATE VIEW reports.estimated_texan_covid_population_by_gross_rent AS
SELECT label AS Gross_Rent, (h.percent/100)*
                                (((SELECT AVG(incident_rate)
                                    FROM datamart.cases 
                                    WHERE last_update=(SELECT MAX(last_update) 
                                                        FROM datamart.cases AS c 
                                                        JOIN datamart.locations_Beam AS l ON c.location_id=l.id 
                                                        WHERE l.state='Texas' AND l.country='US'))/100000)
                                *
                                ((SELECT estimate FROM datamart.demographics WHERE label='Total population')*8.74/100)) AS Estimated_COVID_Affected_Population 
FROM datamart.housing AS h
WHERE subcategory='gross rent' AND percent IS NOT null
ORDER BY Gross_Rent

""


In [30]:
%%bigquery
CREATE VIEW reports.estimated_confirmed_cases_by_education AS
SELECT label AS Enrollment, (s.percent/100)*
                                (SELECT SUM(Confirmed)
                                    FROM datamart.cases
                                     WHERE last_update between '2021-01-01' AND '2021-01-31') AS Estimated_COVID_Confirmed_Cases
FROM datamart.social AS s
WHERE (subcategory='School Enrollment' OR subcategory = 'Educational Attainment') AND percent IS NOT null
ORDER BY Estimated_COVID_Confirmed_Cases

""


In [ ]:
%%bigquery
CREATE VIEW reports.estimated_deaths_by_age_2021 AS
SELECT d.label AS Age_Group, (d.percent/100)*
                                        (SELECT SUM(deaths)
                                        FROM datamart.cases AS c
                                        JOIN datamart.locations AS l ON c.location_id=l.id
                                        WHERE l.country='US' AND last_update between '2021-01-01' and '2021-04-14 23:59:59') AS Estimated_Deaths
                                    
FROM datamart.demographics as d
WHERE (subcategory1 = 'age' AND subcategory2 is null AND d.percent is not null)
ORDER BY Estimated_Deaths DESC LIMIT 10;